In [1]:
import numpy as np, scipy, pandas as pd
from pyDbs.__init__ import *

# Documentation for ```pyDbs.ExcelSymbolLoader```

In [2]:
pd.set_option('display.max_rows', 5)

A small loader for symbol-oriented Excel workbooks. The loader expects a README sheet that lists the sheets to load and the symbol type for each sheet. It builds pandas objects for three symbol types. Each sheet defines a single symbol based on the following structure:

* `set`: Columns indicate index levels. Thus, a single columns is read in as `pd.Index`, 2+ columns are read in as `pd.MultiIndex`.
* `variable`: Variables are read in as `pd.Series`. The values/data for the series is added in a column with name (in first row) matching the name of the symbol. This is a *requirement* for the method to work. All other columns are interpreted as index levels and added as `pd.Index`/`pd.MultiIndex` for the series.
* `scalar`  - a single-column sheet named after the sheet; the first non-NA value is taken as a scalar.

The loader stores all loaded objects in a dictionary keyed by sheet name and also returns a `__meta__` DataFrame with per-sheet metadata (Type, Unit, Index, Description).

## README sheet (minimum columns)
The README sheet must contain at least the following columns:

* `Sheet` - the sheet name in the workbook
* `Type`  - one of `set`, `variable`, or `scalar`

Optional documentation columns that are preserved in `__meta__`: `Unit`, `Index`, `Description`. `Index` is documentation-only and is not used for construction unless `validate_readme_index=True` is passed to the loader.

### README example (tabular)

| Sheet       | Type     | Unit | Index         | Description |
|-------------|----------|------|---------------|-------------|
| Countries   | set      | -    | name          | Country list|
| Region      | variable | -    | country,year  | Region by year |
| DiscountRate| scalar   | pct  |               | Default rate |

## Details - `set` sheets
* One or more columns. Column names become the level names for the resulting index and the order of columns in the sheet determines the level order.
* If the sheet has exactly one column, the loader returns `pd.Index(df[col].dropna(), name=col)`.
* If the sheet has 2 or more columns, the loader returns `pd.MultiIndex.from_frame(df[cols], names=cols)` after dropping rows that are entirely NA.

## Details - `variable` sheets
* The column containing values of the variable must equal the sheet name. All other columns (in sheet order) are treated as index levels.
* The loader constructs a `pd.Series` indexed by the columns that are not the value column, and the series is named after the sheet.

## Details - `scalar` sheets
* A single column sheet whose column name equals the sheet name. The first non-NA cell in the column is returned as a Python scalar (int/float/string, depending on Excel value).

In [3]:
# Example usage - pointing to an example workbook in `data/`
loader = ExcelSymbolLoader("data/ReadExample.xlsx")
data = loader()
# Top-level keys (symbols + '__meta__')
list(data.keys())[:20]

['idxGen',
 'idxF',
 'idxEm',
 'idxCons',
 'pFuel',
 'uEm',
 'VOM',
 'uFuel',
 'taxEm',
 'genCap',
 'mwp',
 'load',
 '__meta__']

The loader returns a dictionary `data` where each key is a sheet name and the values are the constructed pandas objects. The special key `__meta__` is a DataFrame with per-sheet metadata. Typical access patterns:

* `data['MySet']` -> `pd.Index` or `pd.MultiIndex`
* `data['MyVariable']` -> `pd.Series`
* `data['MyScalar']` -> scalar value
* `data['__meta__']` -> `pd.DataFrame` with columns `Type`, `Unit`, `Index`, `Description`.

In [4]:
data.keys()

dict_keys(['idxGen', 'idxF', 'idxEm', 'idxCons', 'pFuel', 'uEm', 'VOM', 'uFuel', 'taxEm', 'genCap', 'mwp', 'load', '__meta__'])

In [5]:
# Example: inspect the metadata and a loaded symbol
loader = ExcelSymbolLoader("data/ReadExample.xlsx")
data = loader()
data['__meta__']
# type(data['VOM'])
# data['VOM']

,Type,Unit,Index,Description
idxGen,set,-,idxGen,Generator technologies
idxF,set,-,idxF,Fuel types
...,...,...,...,...
mwp,variable,€/GJ,idxCons,Marginal willingness to pay
load,variable,GJ,idxCons,Load


## Validation and errors
* The README must contain `Sheet` and `Type` columns — a `ValueError` is raised otherwise.
* If a sheet listed in README is not found in the workbook a `ValueError` is raised.
* If a sheet's `Type` is unknown (not one of `set`, `variable`, `scalar`) a `ValueError` is raised.
* Use `validate_readme_index=True` to ask the loader to check that the columns listed in README.Index are present in the sheet (presence-only check; order is not enforced).

## Notes and tips
* The loader uses the sheet's own column names and order when constructing sets/multiindexes — README.Index is documentation-only unless validation is enabled.
* Empty rows (rows with all-NA in the relevant columns) are dropped before constructing indexes.
* The returned `__meta__` is useful to programmatically discover types and units for downstream processing.

See also: `docs_adj.ipynb`, `docs_Broadcast.ipynb`, and `docs_Gpy.ipynb` for examples of how symbols produced by this loader are used throughout the package.

## Live examples and in-notebook tests
The cells below open the example workbook in `data/` (if present), show the README, load symbols using `ExcelSymbolLoader`, and perform simple assertions that the loaded object types match the README `Type` entries. Run these cells in order inside the notebook.

In [6]:
# Preview workbook and README
import pandas as pd, os
p = 'data/ReadExample.xlsx'
if not os.path.exists(p):
    raise FileNotFoundError(f'Required example workbook not found at {p} - adjust the path or add the file to data/')
xls = pd.ExcelFile(p)
print('sheets =', xls.sheet_names)
if 'README' in xls.sheet_names:
    readme = pd.read_excel(xls, 'README')
    print('README (top rows):')
    display(readme.head())
else:
    print('No README sheet found in workbook; create one as described above.')

sheets = ['README', 'idxGen', 'idxF', 'idxEm', 'idxCons', 'pFuel', 'uEm', 'VOM', 'uFuel', 'taxEm', 'genCap', 'mwp', 'load']
README (top rows):


,Sheet,Type,Unit,Index,Description
0,idxGen,set,-,idxGen,Generator technologies
1,idxF,set,-,idxF,Fuel types
2,idxEm,set,-,idxEm,Emission types
3,idxCons,set,-,idxCons,Consumer classes
4,pFuel,variable,€/GJ,idxF,Fuel price by fuel


In [7]:
# Run the loader and perform basic checks against README
from pyDbs.base import ExcelSymbolLoader
loader = ExcelSymbolLoader(p)
data = loader()
print('Loaded keys (sample):', list(data.keys())[:20])
if '__meta__' in data:
    display(data['__meta__'])
# If README was present, iterate and check types
if 'readme' in globals():
    for _, row in readme.iterrows():
        sheet = str(row['Sheet'])
        typ = str(row['Type']).strip().lower()
        if sheet == 'README':
            continue
        if sheet not in data:
            print(f'Sheet listed in README not loaded: {sheet}')
            continue
        obj = data[sheet]
        ok = False
        try:
            if typ == 'set':
                ok = isinstance(obj, (pd.Index, pd.MultiIndex))
            elif typ == 'variable':
                ok = isinstance(obj, pd.Series)
            elif typ == 'scalar':
                ok = not isinstance(obj, (pd.Series, pd.Index, pd.DataFrame))
            else:
                print(f'Unknown README.Type: {typ} for sheet {sheet}')
            print(f'{sheet}: expected {typ:8} -> loaded {type(obj).__name__:12} :', 'OK' if ok else 'MISMATCH')
        except Exception as e:
            print(f'Error checking {sheet}:', e)

Loaded keys (sample): ['idxGen', 'idxF', 'idxEm', 'idxCons', 'pFuel', 'uEm', 'VOM', 'uFuel', 'taxEm', 'genCap', 'mwp', 'load', '__meta__']


,Type,Unit,Index,Description
idxGen,set,-,idxGen,Generator technologies
idxF,set,-,idxF,Fuel types
...,...,...,...,...
mwp,variable,€/GJ,idxCons,Marginal willingness to pay
load,variable,GJ,idxCons,Load


idxGen: expected set      -> loaded Index        : OK
idxF: expected set      -> loaded Index        : OK
idxEm: expected set      -> loaded Index        : OK
idxCons: expected set      -> loaded Index        : OK
pFuel: expected variable -> loaded Series       : OK
uEm: expected variable -> loaded Series       : OK
VOM: expected variable -> loaded Series       : OK
uFuel: expected variable -> loaded Series       : OK
taxEm: expected variable -> loaded Series       : OK
genCap: expected variable -> loaded Series       : OK
mwp: expected variable -> loaded Series       : OK
load: expected variable -> loaded Series       : OK


In [8]:
# Show one example per type (if present)
def show_example_for_type(data, readme, want_type):
    for _, row in readme.iterrows():
        sheet = str(row['Sheet'])
        typ = str(row['Type']).strip().lower()
        if typ != want_type or sheet == 'README':
            continue
        obj = data.get(sheet)
        print('---', sheet, f'({typ})')
        if want_type == 'set':
            # Index-like object
            print('type:', type(obj))
            try: print('names:', getattr(obj, 'names', None))
            except: pass
            display(obj[:20] if hasattr(obj, '__getitem__') else obj)
        elif want_type == 'variable':
            print('type:', type(obj))
            try: print('index names:', obj.index.names)
            except: pass
            display(obj.head())
        elif want_type == 'scalar':
            print('value:', obj)
        break

if 'readme' in globals():
    show_example_for_type(data, readme, 'set')
    show_example_for_type(data, readme, 'variable')
    show_example_for_type(data, readme, 'scalar')
else:
    print('README not available; run the previous cell to load it.')

--- idxGen (set)
type: <class 'pandas.core.indexes.base.Index'>
names: ['idxGen']


Index(['wind', 'solar', 'nuclear', 'biomass', 'natgas', 'coal'], dtype='object', name='idxGen')

--- pFuel (variable)
type: <class 'pandas.core.series.Series'>
index names: ['idxF']


idxF
coal       2.5
natgas     8.0
biomass    4.0
uranium    0.5
none       0.0
Name: pFuel, dtype: float64

In [9]:
# Inspect all sheets: show columns and a small preview per sheet
import pandas as pd, os
p = 'data/ReadExample.xlsx'
if not os.path.exists(p):
    raise FileNotFoundError(f'Required example workbook not found at {p} - add the file to data/')
xls = pd.ExcelFile(p)
print('sheets =', xls.sheet_names)
for sheet in xls.sheet_names:
    if sheet == 'README':
        continue
    df = pd.read_excel(xls, sheet_name=sheet)
    print('Sheet:', sheet)
    print(' columns ->', list(df.columns))
    display(df.head())

sheets = ['README', 'idxGen', 'idxF', 'idxEm', 'idxCons', 'pFuel', 'uEm', 'VOM', 'uFuel', 'taxEm', 'genCap', 'mwp', 'load']
Sheet: idxGen
 columns -> ['idxGen']


,idxGen
0,wind
1,solar
2,nuclear
3,biomass
4,natgas


Sheet: idxF
 columns -> ['idxF']


,idxF
0,coal
1,natgas
2,biomass
3,uranium
4,none


Sheet: idxEm
 columns -> ['idxEm']


...

,idxGen,genCap
0,wind,720
1,solar,360
2,nuclear,3600
3,biomass,180
4,natgas,1440


Sheet: mwp
 columns -> ['idxCons', 'mwp']


,idxCons,mwp
0,residential,30
1,industrial,25


Sheet: load
 columns -> ['idxCons', 'load']


,idxCons,load
0,residential,5000
1,industrial,8000


In [10]:
# Sheet-by-sheet demonstration: show how the loader constructs each object
from pyDbs.base import ExcelSymbolLoader
loader = ExcelSymbolLoader(p)
data = loader()
meta = data.get('__meta__') if '__meta__' in data else None
if meta is not None:
    display(meta)
# iterate README if present, otherwise iterate meta index
if 'readme' in globals():
    rows = readme.itertuples(index=False)
else:
    rows = ((name, dict(Type=row['Type'])) for name,row in (data['__meta__'].iterrows() if '__meta__' in data else []))
for r in rows:
    # r may be namedtuple from readme or (name, dict) from meta fallback
    try:
        sheet = str(r.Sheet)
        typ = str(r.Type).strip().lower()
    except Exception:
        sheet = r[0] if isinstance(r, tuple) else str(r)
        typ = (r[1].get('Type') if isinstance(r[1], dict) else None) if isinstance(r, tuple) else None
        typ = str(typ).strip().lower() if typ is not None else 'unknown'
    if sheet == 'README':
        continue
    print(f'Sheet: {sheet}   (Type: {typ})')
    raw = pd.read_excel(xls, sheet_name=sheet)
    print(' raw columns:', list(raw.columns))
    # Show loader result
    obj = data.get(sheet)
    print(' loader returned type:', type(obj))
    # Show how to build it manually (demonstration)
    cols = list(raw.columns)
    df_clean = raw.dropna(how='all', subset=cols)
    if typ == 'set':
        if len(cols) == 1:
            print(' Construction: pd.Index(df_clean["{0}"].dropna(), name="{0}")'.format(cols[0]))
            display(pd.Index(df_clean[cols[0]].dropna(), name=cols[0]))
        else:
            print(' Construction: pd.MultiIndex.from_frame(df_clean[cols], names=cols)')
            display(pd.MultiIndex.from_frame(df_clean[cols], names=cols).to_frame(index=False).head())
    elif typ == 'variable':
        value_col = sheet if sheet in cols else cols[-1]
        index_cols = [c for c in cols if c != value_col]
        print(' Interpreting value column ->', value_col)
        print(' Index columns ->', index_cols)
        if index_cols:
            s = df_clean.set_index(index_cols)[value_col]
            display(s.head())
        else:
            print(' No index columns found; raw values:')
            display(df_clean[value_col].head())
    elif typ == 'scalar':
        col = sheet if sheet in cols else cols[0]
        val = df_clean[col].dropna().iloc[0] if df_clean[col].dropna().shape[0] > 0 else None
        print(' Scalar (first non-NA) ->', val)
        print(' loader returned ->', obj)
    else:
        print(' Unknown type, raw preview:')
        display(df_clean.head())

,Type,Unit,Index,Description
idxGen,set,-,idxGen,Generator technologies
idxF,set,-,idxF,Fuel types
...,...,...,...,...
mwp,variable,€/GJ,idxCons,Marginal willingness to pay
load,variable,GJ,idxCons,Load


Sheet: idxGen   (Type: set)
 raw columns: ['idxGen']
 loader returned type: <class 'pandas.core.indexes.base.Index'>
 Construction: pd.Index(df_clean["idxGen"].dropna(), name="idxGen")


Index(['wind', 'solar', 'nuclear', 'biomass', 'natgas', 'coal'], dtype='object', name='idxGen')

Sheet: idxF   (Type: set)
 raw columns: ['idxF']
 loader returned type: <class 'pandas.core.indexes.base.Index'>
 Construction: pd.Index(df_clean["idxF"].dropna(), name="idxF")


Index(['coal', 'natgas', 'biomass', 'uranium', 'none'], dtype='object', name='idxF')

...

idxGen
wind        720
solar       360
nuclear    3600
biomass     180
natgas     1440
Name: genCap, dtype: int64

Sheet: mwp   (Type: variable)
 raw columns: ['idxCons', 'mwp']
 loader returned type: <class 'pandas.core.series.Series'>
 Interpreting value column -> mwp
 Index columns -> ['idxCons']


idxCons
residential    30
industrial     25
Name: mwp, dtype: int64

Sheet: load   (Type: variable)
 raw columns: ['idxCons', 'load']
 loader returned type: <class 'pandas.core.series.Series'>
 Interpreting value column -> load
 Index columns -> ['idxCons']


idxCons
residential    5000
industrial     8000
Name: load, dtype: int64